In [1]:
import pandas as pd 
import numpy as np
from dash_website.utils.graphs.add_line_and_annotation import add_line_and_annotation
from dash_website.correlation_between import CUSTOM_ORDER

order_by = "custom"
data_genetics_heritability = pd.read_feather(f"../../all_data/genetics/heritability/heritability.feather").to_dict()

In [31]:
import plotly.graph_objs as go

heritability = pd.DataFrame(data_genetics_heritability).set_index(["dimension", "subdimension"])

if order_by == "h2":
    sorted_dimensions = heritability.sort_values(by="h2", ascending=False).index
    
    sorted_heritability = heritability.loc[sorted_dimensions]

    bars = go.Bar(
                x=[" - ".join(elem) for elem in sorted_dimensions.values], #[sorted_dimensions.get_level_values("dimension"), sorted_dimensions.get_level_values("subdimension")],
                y=sorted_heritability["h2"],
                error_y={"array": sorted_heritability["h2_std"], "type": "data"},
                name="Heritability",
                marker_color="indianred",
            )
    fig.update_layout(font={"size": 8})

else:  # order_by == "custom"
    sorted_dimensions = heritability.loc[pd.Index(CUSTOM_ORDER).drop(["*", "*instances01"])].index

    sorted_heritability = heritability.loc[sorted_dimensions]

    bars = go.Bar(
                x=np.arange(5, 10 * sorted_heritability.shape[0] + 5, 10),
                y=sorted_heritability["h2"],
                error_y={"array": sorted_heritability["h2_std"], "type": "data"},
                name="Heritability",
                marker_color="indianred",
            )

    fig = go.Figure(bars)

    fig.update_layout(
        xaxis={
            "tickvals": np.arange(5, 10 * sorted_heritability.shape[0] + 5, 10),                
            "ticktext": [" - ".join(elem) for elem in sorted_dimensions.values],
            },
    )

    dimensions = (
        sorted_heritability.index.to_frame()[["dimension", "subdimension"]]
        .reset_index(drop=True)
    )
    dimensions["position"] = fig["layout"]["xaxis"]["tickvals"]
    dimensions.set_index(["dimension", "subdimension"], inplace=True)

    lines = []
    annotations = []

    for dimension in dimensions.index.get_level_values("dimension").drop_duplicates():
        dimension_inner_margin = -0.25
        dimension_outer_margin = -0.5

        min_position = dimensions.loc[dimension].min()
        max_position = dimensions.loc[dimension].max()

        line, annotation = add_line_and_annotation(
            dimension,
            "x",
            "y",
            min_position,
            max_position,
            dimension_inner_margin,
            dimension_outer_margin,
            90,
            10,
        )

        lines.append(line)
        annotations.append(annotation)

        for subdimension in dimensions.loc[dimension].index.get_level_values("subdimension").drop_duplicates():
            subdimension_inner_margin = 0
            subdimension_outer_margin = -0.25

            submin_position = dimensions.loc[(dimension, subdimension)].min()
            submax_position = dimensions.loc[(dimension, subdimension)].max()

            line, annotation = add_line_and_annotation(
                subdimension,
                "x",
                "y",
                submin_position,
                submax_position,
                subdimension_inner_margin,
                subdimension_outer_margin,
                90,
                8,
            )

            lines.append(line)
            annotations.append(annotation)

    # The final top/right line
    line, _ = add_line_and_annotation(
        dimension,
        "x",
        "y",
        min_position,
        max_position,
        0,
        dimension_outer_margin,
        0,
        10,
        final=True,
    )

    lines.append(line)

    fig["layout"]["shapes"] = lines
    fig["layout"]["annotations"] = annotations
    fig.update_layout(xaxis={"showticklabels": False})

fig.update_layout(
    yaxis={"showgrid": False, "zeroline": False},
    xaxis={"showgrid": False, "zeroline": False},
)
fig